In [1]:
# ========================================================================
# 1. IMPORTAÇÕES E CARREGAMENTO DOS DADOS
# ========================================================================
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import os
from tensorflow.keras.models import load_model
import joblib

# CARREGAR AS BASES (ÚNICA coisa que precisa do treinamento)
url_daily = "https://media.githubusercontent.com/media/ruanvirginio/masters/refs/heads/main/bases_tratadas/daily_peak_transformers_dataset.csv"
df_daily = pd.read_csv(url_daily, sep=';', encoding='latin-1')

print("✅ Dados carregados com sucesso!")
print(f"Shape: {df_daily.shape}")
print(f"Transformadores únicos: {df_daily['id'].unique()}")

# ========================================================================
# 2. CARREGAR OS MODELOS SALVOS
# ========================================================================
trafos = ['T1', 'T2']
modelos = ['SVR', 'LSTM', 'CNN', 'CNN_LSTM']
base_dir = r"G:\Meu Drive\Estudos\Mestrado\Github\masters\mestrado\modelos"

modelos_carregados = {}
for trafo in trafos:
    for modelo in modelos:
        extensao = "h5" if modelo in ["LSTM", "CNN", "CNN_LSTM"] else "pkl"
        caminho = os.path.join(base_dir, f"{modelo}_{trafo}.{extensao}")
        
        if os.path.exists(caminho):
            if modelo in ["LSTM", "CNN", "CNN_LSTM"]:
                modelos_carregados[(modelo, trafo)] = load_model(caminho)
            else:
                modelos_carregados[(modelo, trafo)] = joblib.load(caminho)
            print(f"✅ Modelo carregado: {caminho}")
        else:
            print(f"⚠️ Arquivo não encontrado: {caminho}")

# ========================================================================
# 3. FUNÇÕES PARA TESTE (MESMAS DO CÓDIGO ANTERIOR)
# ========================================================================
def preparar_dados_teste(df_daily, trafo, janela=365):
    """Prepara dados de teste na MESMA estrutura usada no treinamento"""
    print(f"\n📊 Preparando dados de teste para {trafo}...")
    
    df_trafo = df_daily[df_daily['id'] == trafo].copy()
    df_trafo = df_trafo[['datahora', 'S']].set_index('datahora').sort_index()
    
    print(f"   Dados disponíveis para {trafo}: {len(df_trafo)} registros")
    
    # Separar os últimos 30% para teste
    split_idx = int(len(df_trafo) * 0.7)
    df_test = df_trafo.iloc[split_idx:]
    
    print(f"   Dados de teste: {len(df_test)} registros")
    
    # Criar janelas deslizantes
    X_test, y_test = [], []
    for i in range(janela, len(df_test)):
        X_test.append(df_test.iloc[i-janela:i].values)
        y_test.append(df_test.iloc[i, 0])
    
    X_test = np.array(X_test)
    y_test = np.array(y_test)
    
    print(f"   Shape final X_test: {X_test.shape}")
    
    return X_test, y_test, df_test.index[janela:]

def testar_modelo(modelo_nome, trafo_nome, X_test, y_test, datas_test):
    """Testa um modelo específico com dados preparados"""
    print(f"\n🧪 Testando {modelo_nome} - {trafo_nome}...")
    
    modelo = modelos_carregados.get((modelo_nome, trafo_nome))
    if modelo is None:
        print(f"   ❌ Modelo não encontrado!")
        return None
    
    try:
        # Fazer previsões
        if modelo_nome == 'SVR':
            X_test_flat = X_test.reshape(X_test.shape[0], -1)
            y_pred = modelo.predict(X_test_flat)
        else:
            y_pred = modelo.predict(X_test).flatten()
        
        # Calcular métricas
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        rmse = np.sqrt(mse)
        
        print(f"   ✅ Previsões realizadas!")
        print(f"   📊 Métricas:")
        print(f"      - RMSE: {rmse:.4f}")
        print(f"      - MAE:  {mae:.4f}")
        print(f"      - R²:   {r2:.4f}")
        
        return {
            'y_real': y_test,
            'y_pred': y_pred,
            'datas': datas_test,
            'metrics': {'RMSE': rmse, 'MAE': mae, 'R2': r2}
        }
        
    except Exception as e:
        print(f"   ❌ Erro ao testar modelo: {e}")
        return None

# ========================================================================
# 4. EXECUTAR TESTES
# ========================================================================
print("\n" + "="*60)
print("🚀 INICIANDO TESTES DOS MODELOS CARREGADOS")
print("="*60)

# Testar cada modelo disponível
for trafo in trafos:
    print(f"\n{'='*50}")
    print(f"🎯 TRANSFORMADOR: {trafo}")
    print(f"{'='*50}")
    
    # Preparar dados de teste para este trafo
    X_test, y_test, datas_test = preparar_dados_teste(df_daily, trafo, janela=365)
    
    # Testar cada modelo disponível para este trafo
    for modelo_nome in modelos:
        if (modelo_nome, trafo) in modelos_carregados and modelos_carregados[(modelo_nome, trafo)] is not None:
            resultado = testar_modelo(modelo_nome, trafo, X_test, y_test, datas_test)

✅ Dados carregados com sucesso!
Shape: (218337, 3)
Transformadores únicos: ['T1' nan 'T10' 'T11a' 'T11b' 'T12' 'T13a' 'T13b' 'T14' 'T15' 'T16a'
 'T16b' 'T17' 'T18' 'T19a' 'T19b' 'T2' 'T20a' 'T20b' 'T21a' 'T21b' 'T22a'
 'T22b' 'T23' 'T24' 'T25' 'T26' 'T27' 'T28a' 'T28b' 'T29' 'T3' 'T30a'
 'T30b' 'T31' 'T32' 'T33' 'T34' 'T35' 'T36a' 'T36b' 'T37' 'T38' 'T39' 'T4'
 'T40' 'T41' 'T42a' 'T42b' 'T43' 'T44' 'T45' 'T46a' 'T46b' 'T47' 'T48'
 'T49' 'T5' 'T50' 'T51' 'T52a' 'T52b' 'T52c' 'T53' 'T54' 'T55' 'T56' 'T57'
 'T58' 'T59' 'T6' 'T60a' 'T60b' 'T61' 'T62' 'T63' 'T64' 'T65' 'T66' 'T67'
 'T68a' 'T68b' 'T69a' 'T69b' 'T70a' 'T70b' 'T71' 'T72' 'T7a' 'T7b' 'T8'
 'T9']
✅ Modelo carregado: G:\Meu Drive\Estudos\Mestrado\Github\masters\mestrado\modelos\SVR_T1.pkl
⚠️ Arquivo não encontrado: G:\Meu Drive\Estudos\Mestrado\Github\masters\mestrado\modelos\LSTM_T1.h5
⚠️ Arquivo não encontrado: G:\Meu Drive\Estudos\Mestrado\Github\masters\mestrado\modelos\CNN_T1.h5
⚠️ Arquivo não encontrado: G:\Meu Drive\Estudo